<a href="https://colab.research.google.com/github/muhuuh/finetuning/blob/main/fine_tuning_unsloth.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-ramqawnp/unsloth_ac8160b93b36446385f61f0d53b0e6e9
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-ramqawnp/unsloth_ac8160b93b36446385f61f0d53b0e6e9
  Resolved https://github.com/unslothai/unsloth.git to commit e0bca1692a72bbf00ac5e3d38b30e241b1c4fb28
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.8/506.8 kB 42.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 kB 28.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.2/181.2 kB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.6/47.6 MB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 99.7 MB/s eta 0:00:00


In [2]:
import os
os.environ["TORCHDYNAMO_DISABLE"] = "1"

In [3]:
from unsloth import FastLanguageModel

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = 'unsloth/Phi-4-mini-instruct-unsloth-bnb-4bit',
    max_seq_length = 2048,
    dtype = None,
    load_in_4bit = True
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2026.1.4: Fast Phi3 patching. Transformers: 4.57.6.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/3.23G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/171 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/15.5M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/282 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/238 [00:00<?, ?B/s]

In [5]:
import json
from datasets import Dataset

with open("./sample_data/people_data.json", "r", encoding="utf-8") as f:
    data = json.load(f)

ds = Dataset.from_list(data)

def to_text(ex):
    resp = ex["response"]
    if not isinstance(resp, str):
        resp = json.dumps(resp, ensure_ascii=False)
    msgs = [
        {"role": "user", "content": ex["prompt"]},
        {"role": "assistant", "content": resp},
    ]
    return {
        "text": tokenizer.apply_chat_template(
            msgs, tokenize=False, add_generation_prompt=False
        )
    }

dataset = ds.map(to_text, remove_columns=ds.column_names)

Map:   0%|          | 0/300 [00:00<?, ? examples/s]

In [6]:
# Config From GitHub (without seed)
model = FastLanguageModel.get_peft_model(
    model,
    r = 64,  # rank of matrices (for LoRA)
    target_modules=[
        'q_proj', 'k_proj', 'v_proj', 'o_proj',
        'gate_proj', 'up_proj', 'down_proj',
    ],  # which layers to inject LoRA into
    lora_alpha = 64 * 2,  # scaling factor, usually 2x rank
    lora_dropout = 0,  # no dropout, increase for regularizaiton
    bias = 'none',  # bias stays frozen, only learn the low-rank matrices
    use_gradient_checkpointing = 'unsloth',  # activate custom checkpointing scheme of Unsloth -> higher compute but less GPU memory when backpropagating
)

Unsloth: Making `model.base_model.model.model` require gradients


In [7]:
from trl import SFTTrainer, SFTConfig
from transformers import TrainingArguments
import torch

torch._dynamo.config.suppress_errors = True

trainer = SFTTrainer(  # supervised fine-tuning trainer
    model = model,
    train_dataset = dataset,
    tokenizer = tokenizer,
    dataset_text_field = 'text',
    max_seq_length = 2048,
    args = SFTConfig(
        per_device_train_batch_size = 2,  # each GPU reads 2 tokenized sequences at once
        gradient_accumulation_steps = 4,  # accumulate loss for 4 iterations before optimizer step -> effective batch 2 * 4 = 8
        warmup_steps = 10,  # linearly "climb" to the learning rate from 0 in the first 10 steps
        max_steps = 60,  # max steps before stopping (unless epochs out before that)
        logging_steps = 1,  # log every single step
        output_dir = "outputs",  # where to store checkpoints, logs etc.
        optim = "adamw_8bit",  # 8-bit AdamW optimizer
        num_train_epochs = 3  # number of epochs, unless we reach 60 steps first
    ),
)

trainer.train()

Unsloth: Tokenizing ["text"] (num_proc=4):   0%|          | 0/300 [00:00<?, ? examples/s]

The model is already on multiple devices. Skipping the move to device specified in `args`.


🦥 Unsloth: Padding-free auto-enabled, enabling faster training.


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 300 | Num Epochs = 2 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 35,651,584 of 3,871,673,344 (0.92% trained)


Step,Training Loss
1,3.648800
2,3.495600
3,3.377800
4,3.471500
5,3.511000
6,3.214000
7,3.272800
8,3.097600
9,3.060700
10,2.808600


TrainOutput(global_step=60, training_loss=1.7826090236504872, metrics={'train_runtime': 133.5739, 'train_samples_per_second': 3.594, 'train_steps_per_second': 0.449, 'total_flos': 565636974280704.0, 'train_loss': 1.7826090236504872, 'epoch': 1.5866666666666667})

In [13]:
#Inference with trained model

FastLanguageModel.for_inference(model)
messages = [
    {
        "role": "user",
        "content": "Mike is 30 years old, loves hiking and works as a coder."
    },
]
# Turn messages to tensor and send to GPU
inputs = tokenizer.apply_chat_template(messages, tokenize=True, add_generation_prompt=True, return_tensors="pt").to("cuda")
# Generate model response with max 512 tokens and 0.7 temperature, smallest set of tokens with cumulative probability of >= 0.9 are kept for random sampling
outputs = model.generate(input_ids=inputs, max_new_tokens=512, use_cache=True, temperature=0.7, do_sample=True, top_p=0.9)
response = tokenizer.batch_decode(outputs)[0]

print(response)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


<|user|>Mike is 30 years old, loves hiking and works as a coder.<|end|><|assistant|>{"age": "30", "gender": "male", "job": "coder", "name": "Mike"}<|end|>


In [16]:
model.save_pretrained_gguf("first_finetuned_model_gguf", tokenizer, quantization_method="q4_k_m", maximum_memory_usage = 0.7)

Unsloth: Merging model weights to 16-bit format...


config.json: 0.00B [00:00, ?B/s]

Found HuggingFace hub cache directory: /root/.cache/huggingface/hub


model.safetensors.index.json: 0.00B [00:00, ?B/s]

Checking cache directory for required files...
Cache check failed: model-00001-of-00002.safetensors not found in local cache.
Not all required files found in cache. Will proceed with downloading.
Checking cache directory for required files...
Cache check failed: tokenizer.model not found in local cache.
Not all required files found in cache. Will proceed with downloading.


...

Unsloth: Preparing safetensor model files: 100%|██████████| 2/2 [02:02<00:00, 61.36s/it]


Note: tokenizer.model not found (this is OK for non-SentencePiece models)


Unsloth: Merging weights into 16bit: 100%|██████████| 2/2 [02:14<00:00, 67.50s/it]


Unsloth: Merge process complete. Saved to `/content/first_finetuned_model_gguf`
Unsloth: Converting to GGUF format...
==((====))==  Unsloth: Conversion from HF to GGUF information
   \\   /|    [0] Installing llama.cpp might take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF f16 might take 3 minutes.
\        /    [2] Converting GGUF f16 to ['q4_k_m'] might take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: Installing llama.cpp. This might take 3 minutes...
Unsloth: Updating system package directories
Unsloth: All required system packages already installed!
Unsloth: Install llama.cpp and building - please wait 1 to 3 minutes
Unsloth: Cloning llama.cpp repository
Unsloth: Install GGUF and other packages
Unsloth: Successfully installed llama.cpp!
Unsloth: Preparing converter script...
Unsloth: [1] Converting model into f16 GGUF format.
This might take 3 minutes...
Unsloth: Initial conversion completed! Files: ['phi-4-mini-instruct.F16.

{'save_directory': 'first_finetuned_model_gguf',
 'gguf_files': ['phi-4-mini-instruct.Q4_K_M.gguf'],
 'modelfile_location': '/content/Modelfile',
 'want_full_precision': False,
 'is_vlm': False,
 'fix_bos_token': False}

In [15]:
print(dataset[0])

{'text': '<|user|>While strolling through a botanical garden, Igor, now 20 earns a living as a tour guide. He is known among friends for conducting amateur astronomy observations in quiet solitude.<|end|><|assistant|>{"age": "20", "gender": "male", "job": "tour guide", "name": "Igor"}<|end|>'}


In [17]:
!zip -r full_model.zip first_finetuned_model_gguf Modelfile phi-4-mini-instruct.Q4_K_M.gguf  # Add any other roots
from google.colab import files
files.download('full_model.zip')

  adding: first_finetuned_model_gguf/ (stored 0%)
  adding: first_finetuned_model_gguf/special_tokens_map.json (deflated 74%)
  adding: first_finetuned_model_gguf/tokenizer_config.json (deflated 82%)
  adding: first_finetuned_model_gguf/chat_template.jinja (deflated 58%)
  adding: first_finetuned_model_gguf/.cache/ (stored 0%)
  adding: first_finetuned_model_gguf/.cache/huggingface/ (stored 0%)
  adding: first_finetuned_model_gguf/.cache/huggingface/.gitignore (stored 0%)
  adding: first_finetuned_model_gguf/.cache/huggingface/download/ (stored 0%)
  adding: first_finetuned_model_gguf/.cache/huggingface/download/model-00001-of-00002.safetensors.lock (stored 0%)
  adding: first_finetuned_model_gguf/.cache/huggingface/download/model.safetensors.index.json.metadata (deflated 26%)
  adding: first_finetuned_model_gguf/.cache/huggingface/download/model.safetensors.index.json.lock (stored 0%)
  adding: first_finetuned_model_gguf/.cache/huggingface/download/tokenizer.model.lock (stored 0%)
  a

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>